In [1]:
import torch

print(f'PyTorch version: {torch.__version__}')
print('*'*10)
print(f'_CUDA version: ')
!nvcc --version
print('*'*10)
print(f'CUDNN version: {torch.backends.cudnn.version()}')
print(f'Available GPU devices: {torch.cuda.device_count()}')
print(f'Device Name: {torch.cuda.get_device_name()}')

PyTorch version: 2.0.1+cu118
**********
_CUDA version: 
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_May__3_19:00:59_Pacific_Daylight_Time_2022
Cuda compilation tools, release 11.7, V11.7.64
Build cuda_11.7.r11.7/compiler.31294372_0
**********
CUDNN version: 8700
Available GPU devices: 1
Device Name: NVIDIA GeForce GTX 1650


In [12]:
#Loading data in Pandas
import pandas as pd
d1 = pd.read_csv("train_patent.csv")

In [13]:
import re

def remove_sentence_with_fig(paragraph):


    # first we remove . after FIG
    paragraph = re.sub(r"FIG.","FIG",paragraph)

    # Split the paragraph into sentences
    sentences = re.split(r'(?<=[.!?])\s+', paragraph)

    # Remove sentences containing "FIG."
    filtered_sentences = [sentence for sentence in sentences if "FIG" not in sentence]

    # Join the remaining sentences to form the updated paragraph
    updated_paragraph = ' '.join(filtered_sentences)

    return updated_paragraph

def clean_text(text):

    # replace ° degree with 
    text = re.sub(r"°", " degree", text)

    # replace US Patent
    text = re.sub(r" U.S. Pat. No. [0-9,]+", "Patent", text)

    # replace semi-colon with full-stop
    text = re.sub(r";", ".", text)
	
    # Remove special characters and symbols
    text = re.sub(r"[^a-zA-Z0-9\s.:]", "", text)
    
    # Normalize whitespace
    text = re.sub(r"\s+", " ", text)
    
    text = text.strip()

    text = remove_sentence_with_fig(text)
    
    return text


In [14]:
#Applying on text column
#Applying Clearning functions oon Dataframe
d1["text"] = d1["text"].apply(clean_text)

In [16]:
# Loading the dataset from dataframe
from datasets import Dataset
dataset = Dataset.from_pandas(d1)


In [17]:
print(dataset)

Dataset({
    features: ['text', 'summary'],
    num_rows: 800
})


In [18]:
# # Loading Local Data
# from datasets import load_dataset
# dataset = load_dataset("csv", data_files="../data/train_patent.csv")

In [20]:
dataset1 = dataset.train_test_split(test_size=0.2)

In [21]:
dataset1

DatasetDict({
    train: Dataset({
        features: ['text', 'summary'],
        num_rows: 640
    })
    test: Dataset({
        features: ['text', 'summary'],
        num_rows: 160
    })
})

In [22]:
dataset1["train"]

Dataset({
    features: ['text', 'summary'],
    num_rows: 640
})

In [23]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [24]:
show_random_elements(dataset1["train"])

In [25]:
# show_random_elements(dataset1["train"])

PreProcessing

In [9]:
# from transformers import AutoTokenizer
# model_checkpoint = "t5-small"
    
# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [10]:
# tokenizer("Hello, this one sentence!")

{'input_ids': [8774, 6, 48, 80, 7142, 55, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [26]:
from transformers import AutoTokenizer, LongT5ForConditionalGeneration


In [17]:
model = (
    LongT5ForConditionalGeneration.from_pretrained("Stancld/longt5-tglobal-large-16384-pubmed-3k_steps")
    .to("cuda")
    .half()
)

d:\UCDPROJECT\Local\venv\Transformerv1\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ACER\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [18]:
tokenizer = AutoTokenizer.from_pretrained("Stancld/longt5-tglobal-large-16384-pubmed-3k_steps")

In [27]:
# !pip install accelerate nvidia-ml-py3

In [28]:
# #Generating GPU Summary
# from pynvml import *


# def print_gpu_utilization():
#     nvmlInit()
#     handle = nvmlDeviceGetHandleByIndex(0)
#     info = nvmlDeviceGetMemoryInfo(handle)
#     print(f"GPU memory occupied: {info.used//1024**2} MB.")


# def print_summary(result):
#     print(f"Time: {result.metrics['train_runtime']:.2f}")
#     print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
#     print_gpu_utilization()

In [29]:
# print_gpu_utilization()

In [23]:
def generate_answers(batch):
    inputs_dict = tokenizer(
        batch["text"], max_length=16384, padding="max_length", truncation=True, return_tensors="pt"
    )
    input_ids = inputs_dict.input_ids.to("cuda")
    attention_mask = inputs_dict.attention_mask.to("cuda")
    output_ids = model.generate(input_ids, attention_mask=attention_mask, max_length=512, num_beams=2)
    batch["predicted_abstract"] = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    return batch

In [ ]:
result = dataset.map(generate_answers, batched=True, batch_size=1)